In [1]:
import os
import re
from dataclasses import dataclass
from time import strftime
from typing import Optional, Dict, Union, Any

import numpy as np
import pandas as pd
from tqdm import tqdm
from sklearn.metrics import roc_auc_score

import datasets
from datasets import Dataset, load_dataset, concatenate_datasets
import evaluate

import torch
import transformers
from transformers import (
    AutoConfig,
    AutoModelForCausalLM,
    AutoModelForSequenceClassification,
    AutoTokenizer,
    DataCollatorForLanguageModeling,
    DataCollatorForSeq2Seq,
    DataCollatorWithPadding,
    GPTNeoXConfig,
    LlamaConfig,
    Seq2SeqTrainer,
    Seq2SeqTrainingArguments,
    T5Config,
    Trainer,
    TrainingArguments,
    logging as hf_logging,
)
from peft import LoraConfig, TaskType, get_peft_model

chrf_metric = evaluate.load("chrf")


In [ ]:
os.environ['CLEARML_API_ACCESS_KEY'] = ''
os.environ['CLEARML_API_SECRET_KEY'] = ''
os.environ['CLEARML_API_HOST'] = 'https://api.clear.ml'
os.environ['CLEARML_LOG_MODEL'] = 'FALSE'
os.environ['CLEARML_TASK'] = 'en-chuvash-post'

In [ ]:
from transformers import AutoTokenizer
import datasets
from datasets import concatenate_datasets

MODEL = 'tencent/HY-MT1.5-7B'  # or 'tencent/HY-MT1.5-7B'

tokenizer = AutoTokenizer.from_pretrained(MODEL)

def build_text_en_chv(example):
    """
    Format English->Chuvash training examples using HY-MT1.5 chat template.
    """
    source_text = example['en']
    target_text = example['chv']
    
    messages = [
        {
            "role": "user",
            "content": f"Translate the following segment into Chuvash, without additional explanation.\n\n{source_text}"
        },
        {
            "role": "assistant",
            "content": target_text
        }
    ]
    
    text = tokenizer.apply_chat_template(
        messages,
        tokenize=False,
        add_generation_prompt=False
    )
    
    example["text"] = text
    return example

def build_text_ru_chv(example):
    """
    Format Russian->Chuvash training examples using HY-MT1.5 chat template.
    """
    source_text = example['ru']
    target_text = example['chv']
    
    messages = [
        {
            "role": "user",
            "content": f"Translate the following segment into Chuvash, without additional explanation.\n\n{source_text}"
        },
        {
            "role": "assistant",
            "content": target_text
        }
    ]
    
    text = tokenizer.apply_chat_template(
        messages,
        tokenize=False,
        add_generation_prompt=False
    )
    
    example["text"] = text
    return example

def tokenize_example(example):
    """
    Tokenize the formatted text for training.
    """
    encoded = tokenizer(
        example["text"],
        truncation=True,
        max_length=512,
        padding=False
    )
    example["input_ids"] = encoded["input_ids"]
    example["attention_mask"] = encoded["attention_mask"]
    
    example["labels"] = encoded["input_ids"].copy()
    
    return example

en_chv_full = datasets.load_dataset('alexantonov/chuvash_english_parallel')['train']
en_chv_split = en_chv_full.train_test_split(test_size=0.05, seed=42)
en_chv_train = en_chv_split['train']
en_chv_test = en_chv_split['test']

en_chv_train = en_chv_train.map(build_text_en_chv, num_proc=8)
en_chv_train = en_chv_train.map(tokenize_example, batched=False, num_proc=16)

en_chv_test = en_chv_test.map(build_text_en_chv, num_proc=8)
en_chv_test = en_chv_test.map(tokenize_example, batched=False, num_proc=16)

print(f"EN-CHV train: {len(en_chv_train)} examples")
print(f"EN-CHV test: {len(en_chv_test)} examples")

ru_chv_full = datasets.load_dataset('alexantonov/chuvash_russian_parallel')['train'].select(range(204_000))
ru_chv_split = ru_chv_full.train_test_split(test_size=0.05, seed=42)
ru_chv_train = ru_chv_split['train']
ru_chv_val = ru_chv_split['test']

ru_chv_train = ru_chv_train.map(build_text_ru_chv, num_proc=4)
ru_chv_train = ru_chv_train.map(tokenize_example, batched=False, num_proc=128)

ru_chv_val = ru_chv_val.map(build_text_ru_chv, num_proc=4)
ru_chv_val = ru_chv_val.map(tokenize_example, batched=False, num_proc=128)

train_combined = concatenate_datasets([en_chv_train, ru_chv_train])
train_combined = train_combined.shuffle(seed=42)

ds = datasets.DatasetDict({
    'train': train_combined,
    'test': en_chv_test
})



ds['test'] = ds['test'].select(range(200))

print(f"\nTrain: {len(ds['train'])} examples (EN+RU -> CHV, shuffled)")
print(f"Test: {len(ds['test'])} examples (EN -> CHV)")

EN-CHV train: 193969 examples
EN-CHV test: 10209 examples

Train: 387769 examples (EN+RU -> CHV, shuffled)
Test: 200 examples (EN -> CHV)


In [ ]:
ds['train'] = ds['train'].remove_columns(['chv', 'en', 'text', 'ru'])
ds['test'] = ds['test'].remove_columns(['chv', 'en', 'text'])

In [ ]:
MODEL = 'tencent/HY-MT1.5-7B'

tokenizer = AutoTokenizer.from_pretrained(MODEL)

if tokenizer.pad_token_id is None:
    tokenizer.pad_token_id = tokenizer.eos_token_id

model = AutoModelForCausalLM.from_pretrained(
    MODEL, 
    dtype=torch.bfloat16, 
).cuda()

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

In [ ]:
from peft import LoraConfig, TaskType, get_peft_model

target_modules = [
    "q_proj",
    "k_proj",
    "v_proj",
    "o_proj",
]

lora_config = LoraConfig(
    task_type=TaskType.CAUSAL_LM,
    inference_mode=False,
    r=16,
    lora_alpha=32,
    lora_dropout=0.05,
    target_modules=target_modules,
)


trainable params: 4,718,592 || all params: 7,509,286,912 || trainable%: 0.0628


In [ ]:
IS_7B = True
if IS_7B:
    ASSISTANT_TOKEN = 127962
else:
    ASSISTANT_TOKEN = 120007

def compute_metrics(eval_preds):
    preds, labels = eval_preds

    if isinstance(preds, tuple):
        preds = preds[0]

    # 2) If preds are logits: (bs, seq, vocab) -> take argmax
    if isinstance(preds, np.ndarray) and preds.ndim == 3:
        preds = np.argmax(preds, axis=-1)

    # 3) Replace -100 (or any negative ids) in BOTH preds and labels
    preds = np.where(preds < 0, tokenizer.pad_token_id, preds)
    labels = np.where(labels < 0, tokenizer.pad_token_id, labels)

    # 4) just to be sure
    preds = preds.astype(np.int32)
    labels = labels.astype(np.int32)

    # 5) Slice to keep only tokens after assistant token (not including it)
    truncated_preds = []
    truncated_labels = []
    
    for pred_seq, label_seq in zip(preds, labels):
        pred_asst_pos = np.where(pred_seq == ASSISTANT_TOKEN)[0]
        if len(pred_asst_pos) > 0:
            start_pos = pred_asst_pos[0] + 1
            truncated_preds.append(pred_seq[start_pos:])
        else:
            truncated_preds.append(pred_seq)
        
        label_asst_pos = np.where(label_seq == ASSISTANT_TOKEN)[0]
        if len(label_asst_pos) > 0:
            start_pos = label_asst_pos[0] + 1
            truncated_labels.append(label_seq[start_pos:])
        else:
            truncated_labels.append(label_seq)

    decoded_preds = tokenizer.batch_decode(truncated_preds, skip_special_tokens=True)
    decoded_labels = tokenizer.batch_decode(truncated_labels, skip_special_tokens=True)

    decoded_preds = [p.strip() for p in decoded_preds]
    decoded_labels = [l.strip() for l in decoded_labels]
    
    print(f'Generated (first 3): {decoded_preds[:10]}')
    print(f'Labels (first 3): {decoded_labels[:10]}')
    
    result = chrf_metric.compute(
        predictions=decoded_preds,
        references=decoded_labels,
        word_order=2
    )
    return {"chrf": result["score"]}


class HYMTSeq2SeqTrainer(Seq2SeqTrainer):

    def _mask_prompt_labels(self, inputs):
        if "labels" not in inputs or "input_ids" not in inputs:
            return inputs

        input_ids = inputs["input_ids"]
        labels = inputs["labels"]
        if not torch.is_tensor(labels):
            return inputs

        labels = labels.clone()

        for i in range(input_ids.size(0)):
            pos = (input_ids[i] == ASSISTANT_TOKEN).nonzero(as_tuple=True)[0]
            if pos.numel() == 0:
                labels[i, :] = -100
            else:
                cut = int(pos[0].item()) + 1
                labels[i, :cut] = -100

        inputs["labels"] = labels
        return inputs

    def compute_loss(self, model, inputs, return_outputs=False, **kwargs):
        inputs = self._mask_prompt_labels(inputs)
        return super().compute_loss(model, inputs, return_outputs=return_outputs, **kwargs)

    def prediction_step(self, model, inputs, prediction_loss_only, ignore_keys=None, **gen_kwargs):
        has_labels = "labels" in inputs
        inputs = self._prepare_inputs(inputs)

        if has_labels:
            inputs = self._mask_prompt_labels(inputs)

        with torch.no_grad():
            if has_labels:
                loss, _ = self.compute_loss(model, inputs, return_outputs=True)
                loss = loss.mean().detach()
            else:
                loss = None

        if prediction_loss_only:
            return (loss, None, None)

        if self.args.predict_with_generate:
            ASSISTANT_TOKEN = 120007
            input_ids = inputs["input_ids"]
            batch_size = input_ids.shape[0]

            truncated_ids = []
            truncated_masks = []

            for i in range(batch_size):
                seq = input_ids[i]
                pos = (seq == ASSISTANT_TOKEN).nonzero(as_tuple=True)[0]
                if pos.numel() > 0:
                    cut = int(pos[0].item()) + 1
                    seq = seq[:cut]
                truncated_ids.append(seq)
                truncated_masks.append(torch.ones_like(seq))

            max_len = max(x.numel() for x in truncated_ids)
            pad_token_id = self.processing_class.pad_token_id

            padded_ids = []
            padded_masks = []
            for ids, mask in zip(truncated_ids, truncated_masks):
                pad_len = max_len - ids.numel()
                if pad_len > 0:
                    ids = torch.cat([ids, torch.full((pad_len,), pad_token_id, dtype=ids.dtype, device=ids.device)])
                    mask = torch.cat([mask, torch.zeros(pad_len, dtype=mask.dtype, device=mask.device)])
                padded_ids.append(ids)
                padded_masks.append(mask)

            gen_inputs = {
                "input_ids": torch.stack(padded_ids),
                "attention_mask": torch.stack(padded_masks),
            }

            gen_kwargs = self._gen_kwargs.copy()
            gen_kwargs = dict(
                max_new_tokens=512,
                num_beams=5,
                do_sample=True,
                num_return_sequences=1,
                repetition_penalty=1.1,
                early_stopping=True,
                temperature=0.7,
                pad_token_id=tokenizer.pad_token_id,
            )
            if gen_kwargs.get("max_length") is None and gen_kwargs.get("max_new_tokens") is None:
                gen_kwargs["max_new_tokens"] = self.args.generation_max_length
            if gen_kwargs.get("num_beams") is None:
                gen_kwargs["num_beams"] = self.args.generation_num_beams

            generated_tokens = model.module.generate(**gen_inputs, **gen_kwargs)
        else:
            generated_tokens = None

        labels = inputs["labels"] if has_labels else None
        return (loss, generated_tokens, labels)



data_collator = DataCollatorForSeq2Seq(
    tokenizer=tokenizer,
    model=model,
    padding=True,
    pad_to_multiple_of=8,
    label_pad_token_id=-100
)

training_args = Seq2SeqTrainingArguments(
    output_dir="checkpoints_7B_lora/en-chv-final",

    num_train_epochs=3,
    per_device_train_batch_size=4,  # max for 80 GB A100
    per_device_eval_batch_size=8,
    gradient_accumulation_steps=1,

    # Learning rate
    learning_rate=1e-4,
    lr_scheduler_type="cosine",
    warmup_ratio=0.05,

    # Optimizer
    optim="adamw_torch",
    weight_decay=0.01,

    # Gradient handling
    max_grad_norm=1.0,

    # Logging
    logging_steps=10,
    logging_dir="./logs",
    logging_strategy="steps",

    # Evaluation
    eval_strategy="steps",
    eval_steps=1_000,

    # Generation for chrf++
    predict_with_generate=True,
    generation_max_length=512,  # Max tokens to generate
    generation_num_beams=5,  # Beam search for better quality

    # Saving
    save_strategy="steps",
    save_steps=1_000,
    save_total_limit=1,
    save_only_model=True,
    save_safetensors=False,
    load_best_model_at_end=False,
    metric_for_best_model="chrf",  # Use chrF++ for best model
    greater_is_better=True,  # Higher chrF is better

    dataloader_num_workers=4,
    dataloader_pin_memory=True,

    # Reproducibility
    seed=42,

    # Reporting
    report_to="clearml",

    remove_unused_columns=True,
    push_to_hub=False,
)


trainer = HYMTSeq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=ds['train'],
    eval_dataset=ds['test'],
    data_collator=data_collator,
    processing_class=tokenizer,
    compute_metrics=compute_metrics
)


trainer.train()

The tokenizer has new PAD/BOS/EOS tokens that differ from the model config and generation config. The model config and generation config were aligned accordingly, being updated with the tokenizer's values. Updated tokens: {'bos_token_id': 127958}.


ClearML Task: overwriting (reusing) task id=445a40860119493a8e6d418b082f39b2
2026-01-08 03:31:53,833 - clearml.Task - INFO - No repository found, storing script code instead
ClearML results page: https://app.clear.ml/projects/c26a6bbc77a549af84da889d0ac9231f/experiments/445a40860119493a8e6d418b082f39b2/output/log
2026-01-08 03:31:56,648 - clearml.Task - WARNING - Parameters must be of builtin type (Transformers/accelerator_config[AcceleratorConfig])


Step,Training Loss,Validation Loss,Chrf
1000,1.037700,1.716513,78.081355
2000,0.840100,1.436051,82.901415
3000,0.767800,1.284212,76.678472
4000,0.687700,1.208644,80.636513
5000,0.677300,1.152494,79.181107
6000,0.639900,1.108040,79.611863
7000,0.653700,1.078570,83.324164
8000,0.647700,1.052249,83.911520
9000,0.643300,1.023414,80.043987
10000,0.599800,1.006340,79.635414


/home/jovyan/.mlspace/envs/bolgov_simson_training/lib/python3.12/site-packages/torch/nn/parallel/_functions.py:71: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(
A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.
A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.
A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.
A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.
A decoder-only architecture is being used, b

Generated (first 3): ['Генерал патне ӑна кӳртмерӗҫ.<|eos|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pa

...

Generated (first 3): ['Генерал патне ӑна кӳртмерӗҫ.<|eos|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pa

'(ReadTimeoutError("HTTPSConnectionPool(host='huggingface.co', port=443): Read timed out. (read timeout=10)"), '(Request ID: b977a981-b62d-4de6-96b8-acce34f2b532)')' thrown while requesting HEAD https://huggingface.co/tencent/HY-MT1.5-7B/resolve/main/config.json
Retrying in 1s [Retry 1/5].
A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.
A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.
A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.
A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.
A decoder-only ar

Generated (first 3): ['Генерал патне ӑна кӳртмерӗҫ.<|eos|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pa

'(ReadTimeoutError("HTTPSConnectionPool(host='huggingface.co', port=443): Read timed out. (read timeout=10)"), '(Request ID: 70c44149-e336-4b8c-86b1-4f45a4aee4b0)')' thrown while requesting HEAD https://huggingface.co/tencent/HY-MT1.5-7B/resolve/main/config.json
Retrying in 1s [Retry 1/5].
'(ReadTimeoutError("HTTPSConnectionPool(host='huggingface.co', port=443): Read timed out. (read timeout=10)"), '(Request ID: ddc03cc7-541a-492a-aec0-da6b2dc62413)')' thrown while requesting HEAD https://huggingface.co/tencent/HY-MT1.5-7B/resolve/main/config.json
Retrying in 1s [Retry 1/5].


TrainOutput(global_step=24236, training_loss=0.6401639572069735, metrics={'train_runtime': 25264.6612, 'train_samples_per_second': 15.348, 'train_steps_per_second': 0.959, 'total_flos': 4.641008847348695e+18, 'train_loss': 0.6401639572069735, 'epoch': 1.0})

In [10]:
trainer.model.save_pretrained('checkpoints_7B_lora/en-chv-final/24000')

'(ReadTimeoutError("HTTPSConnectionPool(host='huggingface.co', port=443): Read timed out. (read timeout=10)"), '(Request ID: 33f733bb-1630-49e7-833e-3fd9ef522fbb)')' thrown while requesting HEAD https://huggingface.co/tencent/HY-MT1.5-7B/resolve/main/config.json
Retrying in 1s [Retry 1/5].
